In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install taichi imageio imageio-ffmpeg

In [ ]:
pip install taichi

### Задание А

In [ ]:
# ДЕКОМПОЗИЦИЯ АНИМАЦИИ (v8.3)

"""
1. Элементы анимации:
   a. Фон: Черный.
   b. Основная форма: Кольцо (N=27 сегментов).
   c. Цвет сегментов: Непрерывная радуга (HSV), распределенная по сегментам. Цвета смещаются при смене выделения.
   d. Анимация 1 (Pop Out): Мгновенное выдвижение/задвижение одного сегмента.
   e. Анимация 2 (Общий поворот): Плавное вращение по часовой стрелке (1 об/30 сек).
   f. Анимация 3 ("Белая Вспышка"): Мгновенная однокадровая вспышка при смене выделения.
   g. Анимация 4 (Выделение): Мгновенная смена выделенного сегмента (CCW, 1/сек).
   h. Анимация 5 (Смещение Цвета): Мгновенное смещение цветового круга HSV на 1 поз. по часовой стрелке при вспышке.
   i. Анимация 6 (Затенение): ***ИЗМЕНЕНО: Плавное затемнение на 20% ширины сегмента со стороны по часовой стрелке (CW), угасающее с удалением от границы.***
   j. Границы: Четкие, черные.

2. Взаимосвязи: Координаты -> Поворот -> Полярные -> Индексы -> Смещение Цвета -> Hue -> Базовый Цвет (HSV->RGB) -> Радиус -> Затенение -> Границы -> Вспышка -> Финальный цвет.

3. Реализация (v8.3):
   - `color_shift_offset = floor(time * h_speed)`.
   - `shifted_segment_idx_f = cast(segment_idx, f32) + color_shift_offset`.
   - `hue = fract(shifted_segment_idx_f / N)`.
   - `base_color = hsv_to_rgb(hue, saturation, value_base)`.
   - `norm_angle_in_segment`.
   - `shading_progress = norm_angle_in_segment / shading_width`
   - `clamped_progress = tm.clamp(shading_progress, 0.0, 1.0)`
   - `shading_factor = tm.mix(shading_strength, 1.0, clamped_progress)`
   - Радиусы: `current_r_inner/outer = mix(base, highlight, is_highlighted)`
   - Границы: Резкий `smoothstep/mix` на основе `min_dist_to_border`, используя `current_r_inner`, `current_r_outer`
   - Остальное как в v8.2 (радиусы, границы, вспышка)
   - `shaded_color = base_color * shading_factor`.
   - `border_mult`.
   - `color_before_flash = mix(border_color, shaded_color, border_mult)`.
   - `is_flash_frame = (floor(time * h_speed) != floor((time - dt) * h_speed))`.
   - If `is_flash_frame`:
     - `shaded_white = vec3(1.0) * shading_factor`.
     - `final_color = mix(border_color, shaded_white, border_mult)`.
   - Else: `final_color = color_before_flash`.
"""

import taichi as ti
import taichi.math as tm
import math
import numpy as np
import imageio # Для сохранения кадров
import os # Для создания папки
import shutil # Для удаления папки перед новым рендером

# --- Функция конвертации цвета HSV в RGB ---
@ti.func
def hsv_to_rgb(h: ti.f32, s: ti.f32, v: ti.f32) -> tm.vec3:
    """
    Преобразует цвет из пространства HSV в RGB.

    Args:
        h (ti.f32): Hue (тон) в диапазоне [0, 1).
        s (ti.f32): Saturation (насыщенность) в диапазоне [0, 1].
        v (ti.f32): Value (яркость) в диапазоне [0, 1].

    Returns:
        tm.vec3: Цвет в пространстве RGB (компоненты в диапазоне [0, 1]).
    """
    h = tm.fract(h) # Гарантируем, что hue в [0, 1)
    r, g, b = 0.0, 0.0, 0.0
    i = ti.floor(h * 6.0); f = h * 6.0 - i; p = v * (1.0 - s)
    q = v * (1.0 - f * s); t = v * (1.0 - (1.0 - f) * s); i = i % 6
    # Выбор компонент RGB в зависимости от сектора hue
    if i == 0: r, g, b = v, t, p
    elif i == 1: r, g, b = q, v, p
    elif i == 2: r, g, b = p, v, t
    elif i == 3: r, g, b = p, q, v
    elif i == 4: r, g, b = t, p, v
    elif i == 5: r, g, b = v, p, q
    # Ограничиваем результат диапазоном [0, 1] для предотвращения артефактов
    return tm.clamp(tm.vec3(r, g, b), 0.0, 1.0)
# --- Конец функции конвертации цвета ---

@ti.data_oriented
class ColorWheelShaderV8_SmoothShade: # Изменено имя класса для ясности
    """
    Класс для генерации анимации цветного кольца с эффектами.

    Плавное угасающее затенение на 20% ширины сегмента.
    Реализует вращающееся кольцо с мгновенно выдвигающимся сегментом,
    однокадровой белой вспышкой при смене сегмента, смещением цветов HSV
    и плавным затенением части сегмента.
    """
    def __init__(self, resolution=(800, 800)):
        """
        Инициализирует шейдер и параметры анимации.

        Args:
            resolution (tuple, optional): Разрешение выходного изображения (ширина, высота).
                                          Defaults to (800, 800).
        """
        # Инициализация Taichi
        try:
            ti.init(arch=ti.gpu)
            print("[Taichi] Инициализация на бэкенде: gpu")
        except Exception:
            try:
                ti.reset()
                ti.init(arch=ti.cpu)
                print("[Taichi] Инициализация на бэкенде: cpu")
            except RuntimeError as e:
                 if "Taichi is already initialized" in str(e):
                      print("[Taichi] Taichi уже инициализирован.")
                 else:
                      raise e

        self.resolution = resolution
        self.pixels = ti.Vector.field(3, dtype=ti.f32, shape=resolution)

        # Параметры геометрии кольца
        self.num_segments = 27
        self.r_inner_base = 0.2
        self.r_inner_highlight = 0.25
        self.r_outer_base = 0.4
        self.r_outer_highlight = 0.5

        # Параметры анимации
        self.highlight_speed = 1.0
        self.global_rotation_speed = 1.0 / 30.0

        # Параметры цвета и границ
        self.saturation = 1.0
        self.value_base = 0.9
        self.shading_strength = 0.2      # Множитель цвета в самой темной точке затенения
        self.shading_width = 0.2         # Доля ширины сегмента для затенения (0.2 = 20%)
        self.border_width = 0.005
        self.border_sharpness = 0.001
        self.border_color = tm.vec3(0.0)

    @ti.kernel
    def main_image(self, time: ti.f32, frame_duration: ti.f32):
        """
        Основная функция рендеринга изображения для заданного кадра времени.

        Args:
            time (ti.f32): Текущее время анимации в секундах.
            frame_duration (ti.f32): Длительность одного кадра в секундах (1.0 / fps).
        """
        res = tm.vec2(self.resolution)
        pi = math.pi
        two_pi = 2.0 * pi

        # Копирование атрибутов класса в локальные переменные ядра
        r_inner_base = self.r_inner_base
        r_inner_highlight = self.r_inner_highlight
        r_outer_base = self.r_outer_base
        r_outer_highlight = self.r_outer_highlight
        num_segments = self.num_segments
        highlight_speed = self.highlight_speed
        global_rotation_speed = self.global_rotation_speed
        saturation = self.saturation
        value_base = self.value_base
        shading_strength = self.shading_strength
        shading_width = self.shading_width # Теперь 0.2
        border_width = self.border_width
        border_sharpness = self.border_sharpness
        border_color = self.border_color

        # Расчет общих параметров кадра
        current_tick = ti.floor(time * highlight_speed)
        highlight_idx = current_tick % num_segments
        color_shift_offset = current_tick
        prev_tick_safe = ti.floor(tm.max(0.0, time - frame_duration) * highlight_speed)
        is_flash_frame = ti.cast(current_tick != prev_tick_safe and time > 1e-6, ti.f32)
        global_rotation_angle = time * global_rotation_speed * two_pi

        for i, j in self.pixels:
            # 1. Нормализация координат пикселя [-0.5, 0.5] и поворот
            uv = (tm.vec2(i, j) - 0.5 * res) / res.y
            c = tm.cos(global_rotation_angle)
            s = tm.sin(global_rotation_angle)
            rot_matrix = tm.mat2([[c, -s], [s, c]])
            rotated_uv = rot_matrix @ uv

            # 2. Преобразование в полярные координаты и определение индекса сегмента
            radius = tm.length(rotated_uv)
            angle = tm.atan2(rotated_uv.y, rotated_uv.x)
            norm_angle_rad = (angle + pi) % two_pi
            norm_angle_01 = norm_angle_rad / two_pi
            segment_idx = ti.floor(norm_angle_01 * num_segments) % num_segments

            # 3. Определение, является ли текущий сегмент пикселя выделенным
            is_segment_highlighted = ti.cast(segment_idx == highlight_idx, ti.f32)

            # 4. Расчет текущих внутреннего и внешнего радиусов
            current_r_inner = tm.mix(r_inner_base, r_inner_highlight, is_segment_highlighted)
            current_r_outer = tm.mix(r_outer_base, r_outer_highlight, is_segment_highlighted)

            # 5. Проверка, находится ли пиксель внутри активной области кольца
            in_active_ring = (radius > current_r_inner) and (radius < current_r_outer)

            final_color = border_color

            if in_active_ring:
                # 6. Расчет цвета сегмента (HSV)
                shifted_segment_idx_f = ti.cast(segment_idx, ti.f32) + color_shift_offset
                hue = tm.fract(shifted_segment_idx_f / num_segments)
                base_segment_color = hsv_to_rgb(hue, saturation, value_base)

                # 7. Расчет фактора затенения (плавное угасание на shading_width ширины с CW стороны)
                segment_angle_width_rad = two_pi / num_segments
                angle_in_segment_rad = norm_angle_rad % segment_angle_width_rad
                norm_angle_in_segment = angle_in_segment_rad / segment_angle_width_rad # [0, 1)

                # Рассчитываем прогресс внутри зоны затенения [0, ...]
                # Если norm_angle_in_segment = 0 (CW край), progress = 0
                # Если norm_angle_in_segment = shading_width (край зоны), progress = 1
                # Если norm_angle_in_segment > shading_width, progress > 1
                shading_progress = norm_angle_in_segment / shading_width

                # Ограничиваем прогресс диапазоном [0, 1]. Все, что дальше края зоны, будет иметь progress = 1.
                clamped_progress = tm.clamp(shading_progress, 0.0, 1.0)

                # Плавно интерполируем фактор затенения:
                # - Если progress=0 (на границе CW), фактор = shading_strength (макс. затемнение)
                # - Если progress=1 (на границе зоны или дальше), фактор = 1.0 (нет затемнения)
                shading_factor = tm.mix(shading_strength, 1.0, clamped_progress)

                # 8. Применение затенения к базовому цвету
                shaded_base_color = base_segment_color * shading_factor

                # 9. Расчет цвета для кадра вспышки
                white_color = tm.vec3(1.0)
                # Применяем тот же фактор плавного затенения к белому цвету вспышки
                shaded_white_color = white_color * shading_factor

                # 10. Выбор между обычным цветом и цветом вспышки
                current_shaded_color = tm.mix(shaded_base_color, shaded_white_color, is_flash_frame)

                # 11. Расчет и применение границ сегмента
                dist_to_inner_rad = radius - current_r_inner
                dist_to_outer_rad = current_r_outer - radius
                radial_dist = tm.min(tm.max(dist_to_inner_rad, 0.0), tm.max(dist_to_outer_rad, 0.0))
                angular_dist_rad = tm.min(angle_in_segment_rad, segment_angle_width_rad - angle_in_segment_rad)
                angular_dist = angular_dist_rad * radius
                min_dist_to_border = tm.min(radial_dist, angular_dist)
                border_mult = tm.smoothstep(border_width - border_sharpness,
                                            border_width + border_sharpness,
                                            min_dist_to_border)

                # Финальное смешивание цвета сегмента с цветом границы
                final_color = tm.mix(border_color, current_shaded_color, border_mult)

            # 12. Запись итогового цвета в поле пикселей
            self.pixels[i, j] = final_color


    def render_to_files(self, duration=30.0, fps=30, output_dir="render_output_v8_smooth"):
        """
        Рендерит анимацию в последовательность PNG файлов.

        Args:
            duration (float, optional): Длительность анимации в секундах. Defaults to 30.0.
            fps (int, optional): Количество кадров в секунду. Defaults to 30.
            output_dir (str, optional): Директория для сохранения кадров.
                                         Defaults to "render_output_v8_smooth".
        """
        if os.path.exists(output_dir):
            print(f"Удаление старой папки: {output_dir}")
            shutil.rmtree(output_dir)
        os.makedirs(output_dir)
        print(f"Создана директория: {output_dir}")

        num_frames = int(duration * fps)
        frame_duration_sec = 1.0 / fps

        print(f"Рендеринг {num_frames} кадров ({duration} сек @ {fps} FPS)...")
        for frame in range(num_frames):
            current_time = float(frame) / fps
            self.main_image(current_time, frame_duration_sec)
            img_np = self.pixels.to_numpy()
            img_np = (np.clip(img_np, 0, 1) * 255).astype(np.uint8)
            filename = os.path.join(output_dir, f"frame_{frame:04d}.png")
            imageio.imwrite(filename, img_np)
            if (frame + 1) % fps == 0 or frame == 0:
                 print(f"Сохранен кадр {frame + 1}/{num_frames} (t={current_time:.2f}с)")
        print("Рендеринг кадров завершен.")
        return output_dir

    def create_video(self, image_folder, output_video_path="animation_v8_smooth.mp4", fps=30):
        """
        Создает видеофайл (.mp4) из последовательности PNG кадров.

        Args:
            image_folder (str): Папка с отрендеренными PNG кадрами.
            output_video_path (str, optional): Путь для сохранения видеофайла.
                                                Defaults to "animation_v8_smooth.mp4".
            fps (int, optional): Количество кадров в секунду для видео. Defaults to 30.
        """
        filenames = sorted([fn for fn in os.listdir(image_folder) if fn.startswith("frame_") and fn.endswith(".png")])
        if not filenames:
            print(f"В папке {image_folder} не найдено PNG файлов для создания видео.")
            return

        print(f"Создание видео '{output_video_path}' из {len(filenames)} кадров с FPS={fps}...")
        writer = imageio.get_writer(output_video_path, fps=fps, codec='libx264', quality=8)
        for filename in filenames:
            file_path = os.path.join(image_folder, filename)
            try:
                frame_data = imageio.imread(file_path)
                writer.append_data(frame_data)
            except Exception as e:
                print(f"Ошибка при обработке кадра {filename}: {e}")
        writer.close()
        print(f"Видео успешно создано: {output_video_path}")

# --- Основной блок для запуска скрипта ---
if __name__ == "__main__":

    # Параметры рендеринга
    render_fps = 25       # Кадры в секунду
    render_duration = 30.0 # Длительность анимации в секундах (18 секунд для полного цикла вращения и смены цветов)
    output_folder_name = "color_wheel_frames_v8_smooth" # Имя папки для кадров
    output_video_name = "color_wheel_animation_v8_smooth.mp4" # Имя видеофайла

    # Создание экземпляра шейдера (используем новый класс)
    shader = ColorWheelShaderV8_SmoothShade(resolution=(800, 800))

    # 1. Рендеринг кадров анимации в файлы
    output_folder = shader.render_to_files(duration=render_duration, fps=render_fps, output_dir=output_folder_name)

    # 2. Создание видеофайла из отрендеренных кадров
    if output_folder and os.path.exists(output_folder):
        shader.create_video(image_folder=output_folder, output_video_path=output_video_name, fps=render_fps)
    else:
        print(f"Папка с кадрами '{output_folder_name}' не найдена или рендеринг не был выполнен. Видео не будет создано.")

[Taichi] Starting on arch=cuda
[Taichi] Инициализация на бэкенде: gpu
Удаление старой папки: color_wheel_frames_v8_smooth
Создана директория: color_wheel_frames_v8_smooth
Рендеринг 750 кадров (30.0 сек @ 25 FPS)...
Сохранен кадр 1/750 (t=0.00с)
Сохранен кадр 25/750 (t=0.96с)
Сохранен кадр 50/750 (t=1.96с)
Сохранен кадр 75/750 (t=2.96с)
Сохранен кадр 100/750 (t=3.96с)
Сохранен кадр 125/750 (t=4.96с)
Сохранен кадр 150/750 (t=5.96с)
Сохранен кадр 175/750 (t=6.96с)
Сохранен кадр 200/750 (t=7.96с)
Сохранен кадр 225/750 (t=8.96с)
Сохранен кадр 250/750 (t=9.96с)
Сохранен кадр 275/750 (t=10.96с)
Сохранен кадр 300/750 (t=11.96с)
Сохранен кадр 325/750 (t=12.96с)
Сохранен кадр 350/750 (t=13.96с)
Сохранен кадр 375/750 (t=14.96с)
Сохранен кадр 400/750 (t=15.96с)
Сохранен кадр 425/750 (t=16.96с)
Сохранен кадр 450/750 (t=17.96с)
Сохранен кадр 475/750 (t=18.96с)
Сохранен кадр 500/750 (t=19.96с)
Сохранен кадр 525/750 (t=20.96с)
Сохранен кадр 550/750 (t=21.96с)
Сохранен кадр 575/750 (t=22.96с)
Сохранен 

<ipython-input-21-8bc949e50d5a>:322: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frame_data = imageio.imread(file_path)


Видео успешно создано: color_wheel_animation_v8_smooth.mp4
